# Descarga y preparación del corpus de datos

El objetivo de este primer notebook es realizar la carga y preprocesamiento del dataset seleccionado de géneros musicales, para su posterior estudio y entrenamiento mediante modelos de machine learning.

## Importaciones

In [2]:
import librosa
import wave

import numpy as np
import datasets

from tqdm.auto import tqdm
import os

/home/pabloreina/proyecto-aine/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Acceso al conjunto de datos de HuggingFace

El dataset que usaremos para este trabajo se ha obtenido de la librería HuggingFace y recibe el nombre de music genre. En él, nos econtramos con unas 1700 muestras de piezas musicales, en formato mp3 y sampleadas a 22050 Hz. La duración de cada una de ellas varía entre los 270 y 300 segundos. El dataset se encuentra dividio en 3 conjuntos: entrenamiento con 1370 entradas, validación con 171 y test con 172. En cuanto a las categorías, nos encontramos 3 tipos de clasificaciones distitnas para cada dato, desde una más general a otra más concreta. La primera clasificación distingue únicamente las piezas entre música clásica y no clásica. La siguiente hace una distinción entre 9 tipos de géneros musicales distintos y la última y más concreta diferencia entre 16 estilo musicales. Siendo más precisos, la división que se establece en clases es la siguiente:

- 1_Classic
    - 3_Symphony
    - 4_Opera
    - 5_Solo
    - 6_Chamber

- 2_Non_classic
    - 7_Pop
        - 12_Pop_vocal_ballad
        - 13_Adult_contemporary
        - 14_Teen_pop

    - 8_Dance_and_house
        - 15_Contemporary_dance_pop
        - 16_Dance_pop

    - 9_Indie
        - 17_Classic_indie_pop
        - 18_Chamber_cabaret_and_art_pop

    - 10_Soul_or_r_and_b

    - 11_Rock
        - 19_Adult_alternative_rock
        - 20_Uplifting_anthemic_rock
        - 21_Soft_rock
        - 22_Acoustic_pop

In [ ]:
ccmusic_corpus = datasets.load_dataset("ccmusic-database/music_genre", name="default",trust_remote_code=True)

In [ ]:
print(ccmusic_corpus)

In [6]:
print(ccmusic_corpus['train'][0])

{'audio': {'path': 'C:\\Users\\Usuario\\.cache\\huggingface\\datasets\\downloads\\extracted\\9978c1aa27e41c465d1a0a120f523eae02b367b680fff33f401cee81e10d28c4\\audio\\2_non-classic\\11_rock\\21_Soft Rock\\6573efb8158239c2015abeaea6bb8f65.mp3', 'array': array([0., 0., 0., ..., 0., 0., 0.]), 'sampling_rate': 22050}, 'mel': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=496x369 at 0x19C52808110>, 'fst_level_label': 1, 'sec_level_label': 8, 'thr_level_label': 14}


## Generación de la estructura de directorios del corpus

Para procesar nuestro dataset una vez ya cargado deberemos de realizar 3 pasos distintos:

1. **Anotaciones:** Generaremos un csv que nos servirá para identificar la clase de cada una de las entradas, lo que facilitará su posterior estudio y entrenamiento.

2. **Características:** Crearemos otro csv con características extraídas de cada una de las muestras de audio, necesarias para el entrenamiento de modelos de etapas posteriores. Diferenciamos características extraídas del dominio temporal y del frecuencial, que pasamos a detallar a continuación:

- Dominio temporal:
    - **Amplitude Envelope:** Esta característica mide la envolvente de amplitud de la señal de audio, que es una curva suave que representa la variación máxima de la amplitud del audio en el tiempo. Puede ayudar a identificar la forma de la señal, capturando cómo el nivel de sonido cambia a lo largo de una grabación.

    - **Root Mean Square (RMS):** El RMS es una medida de la potencia promedio de la señal de audio. Es efectiva para estimar la "fuerza" o energía de la señal en diferentes puntos en el tiempo, lo cual puede ser útil para tareas como la detección de silencios o la estimación de la dinámica de la señal.

    - **Zero Crossing Rate (ZCR):** La tasa de cruces por cero mide cuántas veces la señal de audio cruza el eje horizontal, es decir, cuántas veces la amplitud pasa de positiva a negativa y viceversa. Esta característica es útil para identificar la naturaleza percusiva de los sonidos y es comúnmente utilizada en la clasificación de géneros musicales y la detección de ritmo.

- Dominio frecuencial:
    - **Band Energy Ratio (BER):** Esta característica calcula la relación entre la energía en dos bandas de frecuencia divididas en un umbral específico. Sirve para medir cómo se distribuye la energía entre los bajos y los altos en una señal de audio, proporcionando una idea clara sobre la presencia de tonos graves frente a agudos.

    - **Spectral Centroid:** El centroide espectral indica el "centro de gravedad" del espectro de frecuencias de una señal. Altos valores sugieren sonidos con presencia de altas frecuencias, mientras que bajos valores indican sonidos más graves. Es útil para caracterizar texturas sonoras.

    - **Spectral Bandwidth:** La banda espectral mide la dispersión del espectro alrededor del centroide espectral. Una banda ancha sugiere un sonido más complejo o con ruido, mientras que una banda estrecha indica un sonido más tonal o puro.

    - **Chroma STFT:** Esta característica extrae el cromagrama de la señal usando la transformada de Fourier de tiempo corto. Los cromagramas representan la intensidad de las diferentes clases de tonos en la música, útiles para el análisis armónico y la identificación de acordes.

    - **Spectral Rolloff:** El rolloff espectral es la frecuencia por debajo de la cual se encuentra un cierto porcentaje de la energía del espectro (comúnmente el 85%). Es una buena medida para determinar el límite superior de las frecuencias presentes en la señal.

    - **Mel-Frequency Cepstral Coefficients (MFCC):** Los MFCC son coeficientes que representan el espectro de frecuencias de una señal en la escala Mel, que aproxima mejor la percepción auditiva humana. Son muy populares en el procesamiento del habla y reconocimiento de audio porque capturan las características básicas del timbre y la textura del sonido.


3. **Audios:** Almacenaremos los archivos de audios en formato wav, lo que permitirá su carga de manera sencilla en el futuro para visualizaciones y tratamiento de los mismos.

* Funciones auxiliares para la generación de características:

In [ ]:
def amplitude_envelope(signal,frame_size=1024,hop_length=512):
    F=frame_size
    H=hop_length
    N=signal.shape[0]
    # OJO: El último bloque no necesariamente tiene F muestras, pero 
    # el Python admite rangos a[i:j] fuera del tamaño del array (devuelve lista vacía)  
    return np.array([max(signal[k:k+F]) for k in range(0, N, H)])


def calculate_ber(signal, split_freq, sample_rate, frame_size=1024, hop_length=512):

    # Compute the spectrogram of the signal
    spec = librosa.stft(signal, n_fft=frame_size, hop_length=hop_length)

    # Calculate the range of frequencies
    range_of_freq = sample_rate / 2
    # Calculate the change in frequency per bin
    change_per_bin = range_of_freq / spec.shape[0]
    # Calculate the bin corresponding to the split frequency
    split_freq_bin = int(np.floor(split_freq / change_per_bin))

    modified_spec = np.abs(spec).T
    res = []
    for sub_arr in modified_spec:
        # Compute the energy in the low-frequency range
        low_freq_density = sum(i ** 2 for i in sub_arr[:split_freq_bin])
        # Compute the energy in the high-frequency range
        high_freq_density = sum(i ** 2 for i in sub_arr[split_freq_bin:])
        # Compute the band energy ratio
        ber_val = low_freq_density / high_freq_density
        res.append(ber_val)
    return np.array(res)

* Función para la generación de las características:

In [5]:
def store_features(audio_data, label_column, csv_file):

    # Extraer la información del audio (array numpy, sample rate, label)
    audio_array = audio_data["audio"]["array"]
    audio_sr = audio_data["audio"]["sampling_rate"]
    audio_label = audio_data[label_column]

    # Extraer las características del audio 

    # Dominio temporal
    envelope = amplitude_envelope(audio_array)
    rms = librosa.feature.rms(y=audio_array)
    zcr = librosa.feature.zero_crossing_rate(audio_array)

    # Dominio frecuencial
    # split_freq a 500 para separar graves de agudos
    ber = calculate_ber(audio_array,500, audio_sr)
    spec_cent = librosa.feature.spectral_centroid(y=audio_array, sr=audio_sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=audio_array, sr=audio_sr)

    chroma_stft = librosa.feature.chroma_stft(y=audio_array, sr=audio_sr)
    rolloff = librosa.feature.spectral_rolloff(y=audio_array, sr=audio_sr)
    mfcc = librosa.feature.mfcc(y=audio_array, sr=audio_sr, n_mfcc=13)

    # Generar fila para introducir en el CSV
    row = f"{audio_label},{np.mean(envelope)},{np.mean(rms)},{np.mean(zcr)},{np.mean(ber)},{np.mean(spec_cent)},{np.mean(spec_bw)},{np.mean(chroma_stft)},{np.mean(rolloff)}"
    for e in mfcc:
        row += f",{np.mean(e)}"

    # Escribir en el CSV
    with open(csv_file, "a") as f:
        f.write(row + "\n")

* Función para almacenar las anotaciones:

In [ ]:
def store_annotation(file_name, audio_data, label_name, label_dict, csv_file):
    label_id = audio_data[label_name]
    label = label_dict[label_id]
    with open(csv_file, "a") as f:
        f.write(f"{file_name},{label_id},{label}\n")

* Función para la generación del fichero wav:

In [ ]:
def store_audio_file(index, audio_data, audio_folder):
    audio_folder = audio_folder + str(index) + ".wav"

    audio_array = audio_data["audio"]["array"]

    ww_obj=wave.open(audio_folder,'w')
    ww_obj.setnchannels(1)
    ww_obj.setsampwidth(2)
    ww_obj.setframerate(22050)

    signal=np.int16(audio_array * 32767)
    ww_obj.writeframesraw(signal)

* Función para la generación del directorio de ficheros del corpus:

In [ ]:
def generate_corpus_files(corpus_data, label_column, corpus_folder):

    # Crear directorio principal
    os.makedirs(corpus_folder, exist_ok=True)

    music_genres = {
        0: "Classic",
        2: "Symphony",
        3: "Opera",
        4: "Solo",
        5: "Chamber",
        1: "Non_classic",
        6: "Pop",
        11: "Pop_vocal_ballad",
        12: "Adult_contemporary",
        13: "Teen_pop",
        7: "Dance_and_house",
        14: "Contemporary_dance_pop",
        15: "Dance_pop",
        8: "Indie",
        16: "Classic_indie_pop",
        17: "Chamber_cabaret_and_art_pop",
        9: "Soul_or_r_and_b",
        10: "Rock",
        18: "Adult_alternative_rock",
        19: "Uplifting_anthemic_rock",
        20: "Soft_rock",
        21: "Acoustic_pop"
    }

    # Almacenar ficheros de cada partición
    for partition in ["train", "validation", "test"]:

        # Crear subdirectorios para almacenar los audios
        os.makedirs(f"{corpus_folder}/{partition}/audios", exist_ok=True)

        # Crear fichero de anotaciones
        with open(f"{corpus_folder}/{partition}/annotations.csv", "w") as f:
            f.write("audio_file,label_id,label_name\n")

        # Crear fichero de características
        with open(f"{corpus_folder}/{partition}/features.csv", "w") as f:
            f.write("label,mean_envelope,mean_rms,mean_zcr,\
                    mean_ber,mean_spec_cent,mean_spec_bw,mean_chroma_stft,mean_rolloff,\
                    mean_mfcc1,mean_mfcc2,mean_mfcc3,mean_mfcc4,mean_mfcc5,mean_mfcc6,mean_mfcc7,mean_mfcc8,mean_mfcc9,mean_mfcc10,mean_mfcc11,mean_mfcc12,mean_mfcc13\n")
            

        for i, audio_data in enumerate(corpus_data[partition]):
            print(audio_data)
            # Guardar características en fichero CSV
            store_features(audio_data, label_column, f"{corpus_folder}/{partition}/features.csv")

            # Guardar anotación en fichero CSV
            store_annotation(i, audio_data, label_column, music_genres, f"{corpus_folder}/{partition}/annotations.csv")
            
            # Guardar el audio en formato WAV
            store_audio_file(i, audio_data, f"{corpus_folder}/{partition}/audios/")

* Carga de datos:

In [ ]:
generate_corpus_files(ccmusic_corpus, "fst_level_label", "ccmusic")

In [ ]:
generate_corpus_files(ccmusic_corpus, "sec_level_label", "ccmusic2")